In [3]:
# Exchange fluxes above the sill
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os

In [4]:
sd = 2 # Sill depth
#/Volumes/Extreme SSD/MITgcm outputs
#/work/oceans/wbao/MITgcm_results/iceplume/fjord_test
#/Users/weiyangbao/.CMVolumes/Caviness-group/iceplume/
file1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_maxhs/state_' + str(format(sd,'03d')) + '.nc')
state1 = file1.isel(T=~file1.get_index("T").duplicated())
grid1 = xr.open_dataset('/work/oceans/wbao/MITgcm_results/iceplume/2_Qsg_maxhs/grid_' + str(format(sd,'03d')) + '.nc')

day = state1.T.data / 86400

vol = grid1.HFacC * grid1.drF * grid1.rA # Z Y X
areaC = grid1.HFacC * grid1.drF * grid1.dyF # Z Y X
areaW = grid1.HFacW * grid1.drF * grid1.dyG # Z Y Xp1
temp = state1.Temp.data
s = state1.S.where(grid1.HFacC>0)
u = state1.U.where(grid1.HFacW>0)
sW = np.where(grid1.HFacC[-1,40,1:] == 0)[-1][0] # Sill west edge index
sE = np.where(grid1.HFacC[-1,40,1:] == 0)[-1][-1] # Sill east edge index
sC = np.ceil((sW+sE)/2).astype(int) # Sill crest index (178)

print('Sill Index: west=' + str(sW) + ',center='+str(sC)+',east='+str(sE))

Sill Index: west=120,center=178,east=235


In [89]:
grid1.drF, grid1.dyF, grid1.HFacC

(<xarray.DataArray 'drF' (Z: 90)>
 array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
        4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.,
        6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6., 6.])
 Coordinates:
   * Z        (Z) float64 -1.0 -3.0 -5.0 -7.0 ... -379.0 -385.0 -391.0 -397.0
 Attributes:
     description:  r cell face separation,
 <xarray.DataArray 'dyF' (Y: 80, X: 300)>
 array([[1000., 1000., 1000., ..., 1000., 1000., 1000.],
        [1000., 1000., 1000., ..., 1000., 1000., 1000.],
        [1000., 1000., 1000., ..., 1000., 1000., 1000.],
        ...,
        [1000., 1000., 1000., ..., 1000., 1000., 1000.],
        [1000., 1000., 1000., ..., 1000., 1000., 1000.],
        [1000., 1000., 1000., ..., 1000., 1000., 1000.]])
 Coordinates:
   * 

In [8]:
uzx = state1.U.data[90:,:,35:45,:].mean(axis=(0,2))
szx = state1.S.data[90:,:,35:45,:].mean(axis=(0,2))
Qw = u[90:,:,35:45,sW].data.mean(0) * areaW[:,35:45,sW].data
Qe = u[90:,:,35:45,sE].data.mean(0) * areaW[:,35:45,sE].data
Fw = s.data[90:,:,35:45,sE].mean(0) * Qw
Fe = s.data[90:,:,35:45,sW].mean(0) * Qe
#Qw[Qw>0].sum()

In [9]:
usE = uzx[:,sE]
usW=uzx[:,sW]
# Layer interface
lW = np.where(usW > 0)[0][0]
lE = np.where(usE > 0)[0][0]
QW = Qw[:lW,:].sum()
QE = Qe[:lE,:].sum()
SW = Fw[:lW,:].sum() / QW
SE = Fe[:lE,:].sum() / QE
print('Volume flux: west=' + str(QW) + ',east='+str(QE)+'; Salinity: west='+str(SW)+',east='+str(SE))

Volume flux: west=-110.25227634236217,east=-180.1861156243831; Salinity: west=32.17263753109608,east=32.32677240796553


In [12]:
Qw[Qw>0].sum(), Qe[Qe>0].sum(), Qw[Qw<0].sum(), Qe[Qe<0].sum()

(14257.582457293027,
 4002.4829217286856,
 -14207.595460063565,
 -3952.5072805374084)

In [85]:

kk = np.where(usE > 0)
np.argwhere(np.diff(kk)>2)[0][-1]

14

In [101]:
# Exclude the upper layer for the TEF calculation
S = state1.S.data
U = (state1.U.data[:,:,:,1:] + state1.U.data[:,:,:,:-1]) / 2
Uzx = U[89:,:,35:45,:].mean(axis=(0,2))
for j in range(Uzx.shape[1]):
    if any(Uzx[:,j]>0)==False:
        pass
    else:
        mid = np.where(Uzx[:,j]>0)[0][0]
        if mid>0:
            S[:,:mid,:,j] = 0
            U[:,:mid,:,j] = 0
        else:
            pass

In [107]:
U[89:,:,35:45,235].mean(axis=(0,2)), S[:,:,35:45,235].mean(axis=(0,2))

(array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         5.7477164e-03,  1.3403377e-02,  2.0338301e-02,  2.6520466e-02,
         3.1679735e-02,  3.5663523e-02,  3.8262829e-02,  3.9335575e-02,
         3.8431920e-02,  3.6073770e-02,  3.2578561e-02,  2.7939562e-02,
         2.1841824e-02,  1.4800218e-02,  6.3113361e-03, -4.4133142e-03,
        -2.0837855e-02, -3.4060717e-02, -3.4333490e-02, -2.6513588e-02,
        -1.8526666e-02, -1.2327636e-02, -7.7841841e-03, -4.4681625e-03,
        -2.3258165e-03, -1.0827372e-03, -4.2545173e-04, -1.3178955e-04,
        -2.5086978e-05, -1.5927246e-06, -9.2033415e-06, -2.3319628e-05,
        -3.4385463e-05, -3.9695187e-05, -3.9943629e-05, -3.6988164e-05,
        -3.2133150e-05, -2.6543745e-05, -2.1183398e-05, -1.6700489e-05,
        -1.3433691e-05, -1.1473621e-05, -1.0741855e-05, -1.0996680e-05,
        -1.1744364e-05, -1.0966130e-05, -9.0068152e-06, -8.5766560e-06,
        -1.0252641e-05, -1.1394770e-05, -1.2031645e-05, -1.22234

In [103]:
Uzx[:,178]

array([-0.02087473, -0.01470359, -0.00803567, -0.00117042,  0.00530886,
        0.0110807 ,  0.01664107,  0.0226366 ,  0.02698811,  0.03029838,
        0.03232051,  0.03264056,  0.03284582,  0.03229995,  0.03074668,
        0.02803979,  0.02423124,  0.01788625,  0.00635953, -0.01030064,
       -0.03050123, -0.11087054,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [104]:
np.where(Uzx[:,178]>0)[0][0]

4